In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [3]:
data=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")

In [4]:
data.drop(["location","keyword",'id'],axis=1,inplace=True)
data.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


# cleaning

In [32]:
def replace_substrings(df):
    replacements = {
        '@\w+': '@user',
        'https?\S+': 'http',
    }
    df["text"] = df["text"].replace(replacements, regex=True)
    return df

In [34]:
data = replace_substrings(data)


# PREPARING


In [35]:
train=data.sample(frac=0.85,random_state=200)
val=data.drop(train.index)

In [36]:
trainY=train.pop('target')
valY=val.pop('target')

In [14]:
from transformers import (BertTokenizerFast,TFBertTokenizer,BertTokenizer,RobertaTokenizerFast,
                          DataCollatorWithPadding,TFRobertaForSequenceClassification,TFBertForSequenceClassification,
                          TFBertModel,create_optimizer)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [37]:
model_id="roberta-base"
tokenizer=RobertaTokenizerFast.from_pretrained(model_id)

In [38]:
tokenized_train_dataset = tokenizer(train['text'].to_list(),padding=True,truncation=True)
tokenized_val_dataset = tokenizer(val['text'].to_list(),padding=True,truncation=True)

In [18]:
# tokenized_dataset = tokenizer('hey there',padding=True,truncation=True,max_length=128)

In [20]:
tokenizer.decode(tokenized_train_dataset['input_ids'][0])

"<s>@DanRyckert @drewscanlon He's blazing through this game with the best stealth skills yet. Nothing beats the silenced M4.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>"

In [39]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(tokenized_train_dataset),
    trainY
))
val_dataset=tf.data.Dataset.from_tensor_slices((
    dict(tokenized_val_dataset),
    valY
))

In [ ]:
# train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
# test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [42]:
train_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(85,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(85,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [46]:
model=TFRobertaForSequenceClassification.from_pretrained(model_id,num_labels=2)
model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Model: "tf_roberta_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 124055040 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 592130    
 ficationHead)                                                   
                                                                 
Total params: 124,647,170
Trainable params: 124,647,170
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
#               loss='binary_crossentropy',
              metrics=['accuracy'])

In [42]:
from tensorflow.keras.callbacks import EarlyStopping
earlyStop=EarlyStopping(monitor='val_loss', patience=3)

In [48]:
model.fit(train_dataset.batch(32),
          epochs=3,
          validation_data=(val_dataset.batch(32)),
          batch_size=32)

Epoch 1/3
203/203 [==============================] - 128s 363ms/step - loss: 0.4614 - accuracy: 0.7891 - val_loss: 0.3964 - val_accuracy: 0.8345
Epoch 2/3
203/203 [==============================] - 60s 294ms/step - loss: 0.3539 - accuracy: 0.8513 - val_loss: 0.4189 - val_accuracy: 0.8266
Epoch 3/3
203/203 [==============================] - 60s 294ms/step - loss: 0.2843 - accuracy: 0.8904 - val_loss: 0.4929 - val_accuracy: 0.8284


In [49]:
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
test.drop(['location','keyword','id'],axis=1,inplace=True)

In [50]:
test.head()

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan


In [51]:
test=replace_substrings(test)

In [52]:
tokenized_test_dataset = tokenizer(test['text'].to_list(),padding=True,truncation=True)

In [53]:
test_dataset=tf.data.Dataset.from_tensor_slices((
    dict(tokenized_test_dataset),
))

In [56]:
predictions=model.predict(test_dataset.batch(32))

102/102 [==============================] - 11s 103ms/step


In [57]:
predictions

TFSequenceClassifierOutput(loss=None, logits=array([[-3.1769493,  2.9393826],
       [-2.6908884,  2.438544 ],
       [-2.9978993,  2.7644324],
       ...,
       [-3.2056253,  2.9662607],
       [-2.6642954,  2.340822 ],
       [-2.7542849,  2.532282 ]], dtype=float32), hidden_states=None, attentions=None)

In [59]:
result = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
result["target"] = tf.argmax(predictions.logits, axis=1).numpy()

In [60]:
result.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [61]:
result.to_csv("/kaggle/working/result.csv", index=False)
